In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [0]:
def isvotes(tag):
    return tag.has_attr('class') and tag.has_attr('onclick') and not tag.has_attr('href') and not tag.has_attr('type')

In [0]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("span", class_= "post__title-text"):
        #title
        info.append(soup.find("span", class_= "post__title-text").text)
        #text
        text = soup.find("div", class_="post__text post__text-html").text
        info.append(cleanhtml(text))
        #date
        b = soup.find("span", class_= "post__time")
        c = b.text.split()[1]
        d = b.text.split()[2]
        if c == 'января':
            a = d + "-" +'01'
            info.append(a)
        elif c == 'ферваля':
            a = d + "-" +'02'
            info.append(a)
        elif c == 'марта':
            a = d + "-" +'03'
            info.append(a)
        elif c == 'апреля':
            a = d + "-" +'04'
            info.append(a)
        elif c == 'мая':
            a = d + "-" +'05'
            info.append(a)
        elif c == 'июня':
            a = d + "-" +'06'
            info.append(a)
        elif c == 'июля':
            a = d + "-" +'07'
            info.append(a)
        elif c == 'августа':
            a = d + "-" +'08'
            info.append(a)
        elif c == 'сентября':
            a = d + "-" +'09'
            info.append(a)
        elif c == 'октября':
            a = d + "-" +'10'
            info.append(a)
        elif c == 'ноября':
            a = d + "-" +'11'
            info.append(a)
        else:
            a = d + "-" +'12'
            info.append(a)
        #hubs
        tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link ")
        hubs = ''
        for i in range(len(tagsshit)):
            hubs += tagsshit[i].text + ' '
        info.append(hubs)
        #tags
        tags = ''
        for tag in soup.find_all('li', class_= "inline-list__item inline-list__item_tag"):
            tags += tag.text + ' '
        info.append(tags)
        #votes
        a = soup.find(isvotes).text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #views
        a = soup.find("div", class_="post-stats__views").text.strip()
        if 'k' in a:
            views = float(a[:-1].replace(',', '.')) * 1000
        else:
            views = int(a)
        info.append(views)
        #bookmarks
        info.append(int(soup.find("span", class_="bookmark__counter js-favs_count").text))
        #comments
        cauthor = soup.find_all('span', class_="user-info__nickname user-info__nickname_small user-info__nickname_comment")
        author = soup.find('span', class_="user-info__nickname user-info__nickname_small").text
        allcauthors = [i.text.split() for i in cauthor]
        comment1 = soup.find_all("div", class_="comment__message")
        comments = [i.text for i in comment1]
        commentss = []
        for i in range(len(comments)):
            if comments[i] != 'НЛО прилетело и опубликовало эту надпись здесь':
                commentss.append(comments[i])
        commentsitog = ''
        for i in range(len(commentss)):
            if allcauthors[i][0] != author:
                commentsitog += commentss[i] + '~#'
        info.append(commentsitog)
    else:
        info.append('NaN')
    print(id)
    return info

In [0]:
i = 239400
for j in range(239500, 351001, 100):
    from multiprocessing import Pool
    with Pool(100) as p:
        b = p.map(document, np.arange(i, j))
    try:
        df = pd.DataFrame(b, columns=['title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
        df.to_feather(str(i) + "-" + str(j) + '.feather')
        from google.colab import files
        files.download(str(i) + "-" + str(j) + '.feather')
        print('done', i, j)
    except:
        print('ooops')
    i += 100

In [ ]:
i = 209000
for j in range(209100, 209401, 100):
    new = pd.read_feather(str(i) + "-" + str(j) + '.feather')
    allhabr = allhabr.append(new, ignore_index=True)
    print('OK')
    i += 100